[Data source ZVV](https://data.stadt-zuerich.ch/dataset/vbz_fahrplandaten_gtfs)

In [17]:
import math
import re

import pandas as pd
import numpy as np

import requests

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns 
sns.set()

import networkx as nx

pd.options.display.float_format = '{:,.8}'.format

In [18]:
df_agency = pd.read_csv("agency.txt")
df_calendar = pd.read_csv("calendar.txt")
df_calendar_dates = pd.read_csv("calendar_dates.txt")
df_routes = pd.read_csv("routes.txt")
df_shapes = pd.read_csv("shapes.txt")
df_stop_times = pd.read_csv("stop_times.txt")
df_stops = pd.read_csv("stops.txt")
df_transfers = pd.read_csv("transfers.txt")
df_trips = pd.read_csv("trips.txt")

**Features Calendar**

In [19]:
df_calendar
df_calendar2 = df_calendar.copy()

df_calendar2["abfahrten_per_week"]=(df_calendar2.monday 
                                    + df_calendar2.tuesday  
                                    + df_calendar2.wednesday  
                                    + df_calendar2.thursday  
                                    + df_calendar2.friday  
                                    + df_calendar2.saturday  
                                    + df_calendar2.sunday 
                                   )
        
df_calendar2["start_date"]= pd.to_datetime(df_calendar2.start_date, format='%Y%m%d')
df_calendar2["end_date"]= pd.to_datetime(df_calendar2.end_date, format='%Y%m%d')

df_calendar2["duration_weeks"] = (df_calendar2.end_date - df_calendar2.start_date).dt.days / 7
df_calendar2["abfahrten_per_year_plan"] = (df_calendar2.duration_weeks * df_calendar2.abfahrten_per_week).map(math.floor)

df_calendar2 = df_calendar2[["service_id"
                            ,"start_date"
                            ,"end_date"
                            ,"abfahrten_per_year_plan"
                            ]]
df_calendar2[:5]

,service_id,start_date,end_date,abfahrten_per_year_plan
0,T3+ez,2017-12-10,2018-12-08,51
1,T3+e8,2017-12-10,2018-12-08,51
2,T2+ac,2017-12-10,2018-12-08,51
3,T2+b0200,2017-12-10,2018-12-08,51
4,T3+r0200,2017-12-10,2018-12-08,51


In [20]:
#count per service_id per exception_type, since I don't care about the actual days
df_calendar_dates2 = df_calendar_dates.groupby(["service_id","exception_type"]).size().reset_index(name = "cnt")
#weighted: 1 means additional ride, 2 means no ride
df_calendar_dates2["cnt_additional_rides"] = np.where(df_calendar_dates2.exception_type == 1, df_calendar_dates2.cnt, -1 * df_calendar_dates2.cnt)
#number of deleted/added rides per service_id
df_calendar_dates3 = df_calendar_dates2.groupby(["service_id"])[["cnt_additional_rides"]].sum().reset_index()

df_calendar_dates3[:5]

,service_id,cnt_additional_rides
0,T0,-10
1,T0+00001,-18
2,T0+00200,-211
3,T0+03200,-17
4,T0+06200,-204


In [21]:
df_calendar3 = df_calendar2.merge(df_calendar_dates3, on = "service_id")
df_calendar3["abfahrten_per_year_actual"] = df_calendar3.abfahrten_per_year_plan + df_calendar3.cnt_additional_rides
# Sanity Check: Does 0 <= days_per_year_actual <= 365 hold?
#df_calendar3.days_per_year_actual.value_counts().reset_index(name = "cnt").sort_values(by="index")
#df_calendar3["ratio_days_per_year"] = df_calendar3.days_per_year_actual / 365
df_calendar3 = df_calendar3[["service_id","abfahrten_per_year_actual"]]

df_calendar_final = df_calendar3.rename({"abfahrten_per_year_actual" : "abfahrten_per_year"}, axis = 1)
df_calendar_final[:5]

,service_id,abfahrten_per_year
0,T3+ez,0
1,T3+e8,0
2,T2+ac,0
3,T2+b0200,44
4,T3+r0200,54


**Features Stops**

In [22]:
#for compatibility with Fahrgastzahlen
def rename_stops(input_name):
    renaming = {
        "Zürich, Altes Krematorium" : "Zürich, Krematorium Sihlfeld"
        ,"Dübendorf, Innovationspark" : "Dübendorf, Eglishölzli"
        ,"Kilchberg ZH, Altersheim" : "Kilchberg ZH, Hallenbad"
    }
    renamed = renaming.get(input_name,None)
    
    return renamed or input_name
    
df_stops2 = df_stops.copy()    
df_stops2["stop_name_actual"] = df_stops2.stop_name.map(rename_stops)

#in Fahrgastzahlen, there is only one pair of coordinates per Haltestelle, not per Haltepunkt. Haltestellen are identified by the same stop_name 
#thus it is prudent to give every Haltepunkt per stop_name the same coordinates - i.e. the mean of the coordinates of the Haltepunkte
df_stops_grouped = df_stops2.groupby(["stop_name_actual"],)[["stop_lat","stop_lon"]].mean().reset_index()
df_stops3 = df_stops2[["stop_id","stop_name_actual"]].merge(df_stops_grouped, on = "stop_name_actual")

df_stops_final = df_stops3[["stop_id","stop_name_actual","stop_lat","stop_lon"]] \
                    .rename({"stop_name_actual" : "stop_name"}, axis = 1)
df_stops_final[:5]

,stop_id,stop_name,stop_lat,stop_lon
0,ch:23026:1167:1:51,"Pfäffikon ZH, Hermatswil",47.394309,8.8262082
1,gen:23026:1167:1:50,"Pfäffikon ZH, Hermatswil",47.394309,8.8262082
2,ch:23026:1198:1:52,"Wiesendangen, Hinterdorf",47.522001,8.7943779
3,gen:23026:1198:1:50,"Wiesendangen, Hinterdorf",47.522001,8.7943779
4,gen:23026:1198:1:51,"Wiesendangen, Hinterdorf",47.522001,8.7943779


**Features Trips**

In [23]:
df_trips[:5]

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id
0,1-25-P-j18-1,T0,167.T0.1-25-P-j18-1.2.R,1-25-P-j18-1.2.R,"Zürich, Bergstation Dolderbahn",1
1,1-25-P-j18-1,T0,168.T0.1-25-P-j18-1.2.R,1-25-P-j18-1.2.R,"Zürich, Bergstation Dolderbahn",1
2,1-25-P-j18-1,T0,169.T0.1-25-P-j18-1.2.R,1-25-P-j18-1.2.R,"Zürich, Bergstation Dolderbahn",1
3,1-25-P-j18-1,T0,170.T0.1-25-P-j18-1.2.R,1-25-P-j18-1.2.R,"Zürich, Bergstation Dolderbahn",1
4,1-25-P-j18-1,T0,171.T0.1-25-P-j18-1.2.R,1-25-P-j18-1.2.R,"Zürich, Bergstation Dolderbahn",1


In [29]:
df_trips2 = df_trips.copy()
df_trips2 = (df_trips2 
                 #How many days per year does the trip occur?
                .merge(df_calendar_final, on = ["service_id"]) 
                 #Where does the trip halt?
                .merge(df_stop_times[["trip_id","stop_id","stop_sequence"]], on = ["trip_id"]) 
                #What are the names/coordinates of the stops?
                .merge(df_stops_final[["stop_id","stop_name","stop_lat","stop_lon"]], on = ["stop_id"]) 
                #What is the colloquial name of the line?
                .merge(df_routes[["route_id","route_short_name"]], on = ["route_id"])
            )
df = df_trips2[["route_short_name","trip_id","trip_headsign","abfahrten_per_year","stop_sequence","stop_name","stop_lat","stop_lon"]] \
                .sort_values(by = ["trip_id","stop_sequence"])
df[:5]

,route_short_name,trip_id,trip_headsign,abfahrten_per_year,stop_sequence,stop_name,stop_lat,stop_lon
733321,2,1.T0.1-2-P-j18-1.18.R,"Zürich,Kalkbreite/Bhf.Wiedikon",0,1,"Zürich, Bahnhof Tiefenbrunnen",47.350239,8.5610935
728285,2,1.T0.1-2-P-j18-1.18.R,"Zürich,Kalkbreite/Bhf.Wiedikon",0,2,"Zürich, Wildbachstrasse",47.353242,8.5587977
728819,2,1.T0.1-2-P-j18-1.18.R,"Zürich,Kalkbreite/Bhf.Wiedikon",0,3,"Zürich, Fröhlichstrasse",47.355263,8.5568753
729353,2,1.T0.1-2-P-j18-1.18.R,"Zürich,Kalkbreite/Bhf.Wiedikon",0,4,"Zürich, Höschgasse",47.357957,8.5545801
729887,2,1.T0.1-2-P-j18-1.18.R,"Zürich,Kalkbreite/Bhf.Wiedikon",0,5,"Zürich, Feldeggstrasse",47.36038,8.5521636


In [30]:
#bring data into format, where every row is a small trip from one to the next stop. 
#I.e rows [(stop1),(stop2),(stop3)] to [(stop1,stop2),(stop2,stop3)]
#do this via self join on the successor of the stop sequence, so for Trip A join stop X-1 and stop X. 
#Precomputing instead of full join & filtering is much faster

df_ = df[["trip_id","stop_sequence","stop_name","stop_lat","stop_lon"]].copy()
df_["stop_sequence_preceding"] = df_.stop_sequence - 1 

df2 = df.merge(df_, left_on = ["trip_id","stop_sequence"], right_on = ["trip_id","stop_sequence_preceding"], suffixes = ("_von","_nach")) \
        .drop(["stop_sequence_preceding"], axis = 1)
    
df2[:10]

,route_short_name,trip_id,trip_headsign,abfahrten_per_year,stop_sequence_von,stop_name_von,stop_lat_von,stop_lon_von,stop_sequence_nach,stop_name_nach,stop_lat_nach,stop_lon_nach
0,2,1.T0.1-2-P-j18-1.18.R,"Zürich,Kalkbreite/Bhf.Wiedikon",0,1,"Zürich, Bahnhof Tiefenbrunnen",47.350239,8.5610935,2,"Zürich, Wildbachstrasse",47.353242,8.5587977
1,2,1.T0.1-2-P-j18-1.18.R,"Zürich,Kalkbreite/Bhf.Wiedikon",0,2,"Zürich, Wildbachstrasse",47.353242,8.5587977,3,"Zürich, Fröhlichstrasse",47.355263,8.5568753
2,2,1.T0.1-2-P-j18-1.18.R,"Zürich,Kalkbreite/Bhf.Wiedikon",0,3,"Zürich, Fröhlichstrasse",47.355263,8.5568753,4,"Zürich, Höschgasse",47.357957,8.5545801
3,2,1.T0.1-2-P-j18-1.18.R,"Zürich,Kalkbreite/Bhf.Wiedikon",0,4,"Zürich, Höschgasse",47.357957,8.5545801,5,"Zürich, Feldeggstrasse",47.36038,8.5521636
4,2,1.T0.1-2-P-j18-1.18.R,"Zürich,Kalkbreite/Bhf.Wiedikon",0,5,"Zürich, Feldeggstrasse",47.36038,8.5521636,6,"Zürich, Kreuzstrasse",47.363128,8.5497651
5,2,1.T0.1-2-P-j18-1.18.R,"Zürich,Kalkbreite/Bhf.Wiedikon",0,6,"Zürich, Kreuzstrasse",47.363128,8.5497651,7,"Zürich, Opernhaus",47.365623,8.54797
6,2,1.T0.1-2-P-j18-1.18.R,"Zürich,Kalkbreite/Bhf.Wiedikon",0,7,"Zürich, Opernhaus",47.365623,8.54797,8,"Zürich, Bellevue",47.36691,8.5452002
7,2,1.T0.1-2-P-j18-1.18.R,"Zürich,Kalkbreite/Bhf.Wiedikon",0,8,"Zürich, Bellevue",47.36691,8.5452002,9,"Zürich, Bürkliplatz",47.366696,8.541032
8,2,1.T0.1-2-P-j18-1.18.R,"Zürich,Kalkbreite/Bhf.Wiedikon",0,9,"Zürich, Bürkliplatz",47.366696,8.541032,10,"Zürich, Börsenstrasse",47.368211,8.5398342
9,2,1.T0.1-2-P-j18-1.18.R,"Zürich,Kalkbreite/Bhf.Wiedikon",0,10,"Zürich, Börsenstrasse",47.368211,8.5398342,11,"Zürich, Paradeplatz",47.369733,8.5389295


In [31]:
df_menge_abfahrten = df.groupby(["stop_name","stop_lat","stop_lon"])[["abfahrten_per_year"]].sum().reset_index() \
                        .sort_values(by="abfahrten_per_year")
    
df_menge_abfahrten[:5]

,stop_name,stop_lat,stop_lon,abfahrten_per_year
2401,"Zürich, Neumühlequai",47.377489,8.543167,0
2260,"Zürich, Garage 9 Hagenholz",47.415032,8.5578275,0
390,"Eglisau, Bahnhof",47.57263,8.5160918,0
2264,"Zürich, Gessnerallee",47.376893,8.5368788,12
1862,"Winterthur, Bahnhof Grüze",47.498361,8.75088,36


In [32]:
df_menge_touren = df2.groupby(["stop_name_von"
                               ,"stop_lat_von"
                               ,"stop_lon_von"
                               ,"stop_name_nach"
                               ,"stop_lat_nach"
                               ,"stop_lon_nach"]
                            )[["abfahrten_per_year"]] \
                            .sum() \
                            .reset_index() \
                            .sort_values(by = "abfahrten_per_year", ascending = False)
            
df_menge_touren[:5]

,stop_name_von,stop_lat_von,stop_lon_von,stop_name_nach,stop_lat_nach,stop_lon_nach,abfahrten_per_year
5496,"Zürich, Löwenplatz",47.375959,8.537319,"Zürich, Bahnhofplatz/HB",47.377127,8.539633,238021
5048,"Zürich, Bürkliplatz",47.366696,8.541032,"Zürich, Bellevue",47.36691,8.5452002,233124
4948,"Zürich, Bellevue",47.36691,8.5452002,"Zürich, Bürkliplatz",47.366696,8.541032,232966
5058,"Zürich, Central",47.37677,8.5437313,"Zürich, Bahnhofplatz/HB",47.377127,8.539633,207124
5108,"Zürich, Escher-Wyss-Platz",47.390489,8.522583,"Zürich, Schiffbau",47.388031,8.5198737,201745


In [33]:
df_menge_abfahrten.to_csv("1a_menge_abfahrten.csv", index = False)
df_menge_touren.to_csv("1a_menge_touren.csv", index = False)